# Optimising Models with TensorBoard

**This lesson is adapted from [sentdex](https://www.youtube.com/watch?v=lV09_8432VA) on YouTube. I have collated this for my own learning experience, as well as for the benefit of others who would like to learn as well. :)**

In this notebook, we will be looking at how to improve our models further by looking at the plotted graphs of our training and validation loss on TensorBoard. 

Many times, the performance of models can be significantly improved by changing for example, the number of layers or number of hidden nodes - basically almost anything. 

Furthermore, it is usualy difficult to predict the optimal number of layers and neurons that your model require without the help of a research paper or continuous trial and error. 

Hence, TensorBoard can help make that process easier by giving you a visual representation of how well your model is doing.

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import numpy as np
import pickle
from tensorflow.keras.callbacks import TensorBoard
import time

Here, we specify the different numbers of dense layers, layer sizes, and convolutional 2D layers that we would like to test. 

For the layer_sizes, the actual numbers that you input actually does not matter much (you can input 30, 60, 120 and it's okay). 

But do take note that inputting extremely large numbers or a large quantity of numbers will evidently take a longer time to train and test.

In [1]:
# dense_layers = [0, 1, 2]
# layer_sizes = [32, 64, 128]
# conv_layers = [1, 2, 3]

dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]

Now, we would like to see what is the total number of different combinations we could possibly go ith. You would see that small changes from above would lead to a large number of different combinations that we could take.

In [3]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

1-conv-32-nodes-0-dense-1591430540
2-conv-32-nodes-0-dense-1591430540
3-conv-32-nodes-0-dense-1591430540
1-conv-64-nodes-0-dense-1591430540
2-conv-64-nodes-0-dense-1591430540
3-conv-64-nodes-0-dense-1591430540
1-conv-128-nodes-0-dense-1591430540
2-conv-128-nodes-0-dense-1591430540
3-conv-128-nodes-0-dense-1591430540
1-conv-32-nodes-1-dense-1591430540
2-conv-32-nodes-1-dense-1591430540
3-conv-32-nodes-1-dense-1591430540
1-conv-64-nodes-1-dense-1591430540
2-conv-64-nodes-1-dense-1591430540
3-conv-64-nodes-1-dense-1591430540
1-conv-128-nodes-1-dense-1591430540
2-conv-128-nodes-1-dense-1591430540
3-conv-128-nodes-1-dense-1591430540
1-conv-32-nodes-2-dense-1591430540
2-conv-32-nodes-2-dense-1591430540
3-conv-32-nodes-2-dense-1591430540
1-conv-64-nodes-2-dense-1591430540
2-conv-64-nodes-2-dense-1591430540
3-conv-64-nodes-2-dense-1591430540
1-conv-128-nodes-2-dense-1591430540
2-conv-128-nodes-2-dense-1591430540
3-conv-128-nodes-2-dense-1591430540


In [4]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))

In [3]:
x = pickle.load(open("x.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

#normalise the data
#since the maximum pixel value is 255, we will divide our x by 255 to achieve values between 0 and 1
x = x/255.0
y = np.array(y)

The code below will be unusually long, but I will try to explain each step as much as I can along the code lines. 

In summary, what we are doing is that we first loop through dense layers, layer sizes and convolutional layers. Do let me know if I am wrong but, there should not be a difference in which order the three factors are looped through first. By doing so, we will run the model through every possible combination that we have seen earlier, and ideally pick the best performing model from TensorBoard.

In [4]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            #identify the name of the combination for easier identificaiton later on
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir = 'logs\{}'.format(NAME))
            print(NAME)
            
            #using the sequential model
            model = Sequential()

            #first layer
            model.add(Conv2D(layer_size, (3, 3), input_shape = x.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2, 2)))
            
            #we will only execute the 2nd and third layers of the convolutional layers when conv_layers > 1
            for l in range(conv_layer - 1):
                #second layer
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2, 2)))
                
            #third layer
            #first we need to flatten the dataset because conv2d pass through a 2D dataset, whereas dense accepts a 1D dataset
            model.add(Flatten())
            
            #we only execute the 1st and 2nd layers of dense layers if dense_layers > 0
            for l in range(dense_layer):
                model.add(Dense(512))
                model.add(Activation('relu'))
                
            #output layer with sigmoid function as activation function
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            #specify the optimizer, loss function and metrics(optional)
            #loss function here is binary crossentropy because we only have 2 outputs (cat or dog)
            model.compile(loss = "binary_crossentropy",
                         optimizer = "adam",
                         metrics = ["accuracy"])

            model.fit(x, y, batch_size = 32, epochs = 10, validation_split = 0.3, callbacks = [tensorboard])

3-conv-64-nodes-1-dense-1591517560
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 18s 1ms/sample - loss: 0.6596 - accuracy: 0.5962 - val_loss: 0.5936 - val_accuracy: 0.6825
Epoch 2/10
17462/17462 [==============================] - 14s 795us/sample - loss: 0.5613 - accuracy: 0.7070 - val_loss: 0.5082 - val_accuracy: 0.7527
Epoch 3/10
17462/17462 [==============================] - 16s 925us/sample - loss: 0.4995 - accuracy: 0.7563 - val_loss: 0.4612 - val_accuracy: 0.7863
Epoch 4/10
17462/17462 [==============================] - 39s 2ms/sample - loss: 0.4455 - accuracy: 0.7929 - val_loss: 0.4634 - val_accuracy: 0.7854
Epoch 5/10
17462/17462 [==============================] - 54s 3ms/sample - loss: 0.4087 - accuracy: 0.8114 - val_loss: 0.4501 - val_accuracy: 0.7910
Epoch 6/10
17462/17462 [==============================] - 54s 3ms/sample - loss: 0.3668 - accuracy: 0.8370 - val_loss: 0.4522 - val_accuracy: 0.7934
Epoch 7/10
17462/1

Next, in order to view your graphs on tensorboard, open your command prompt and navigate to the directory where your logs file is located. Then, input the following command:

*tensorboard --logdir=logs/*

Copy and paste the link outputted into your browser.

Toggle to see all your validation losses to pick the best performing model (it should be the lowest graph). For me, my best performing model was 3-conv-64-nodes-0-dense. It could be a little different for you, but the consensus is that 3 conv layers and 0 dense layers are often the better models.

Next, we should see if we could improve our models better by inputting other different values while keeping the well-performing values constant. So for example, I kept my conv layers at 3, and decide to test out if a larger number of neurons within a dense layer could allow the model to perform better.